In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
df = pd.read_csv("./results/mdi_local.real_data_regression/regression_03_09/varying_sample_row_n/seed331/results.csv")

In [9]:
pd.set_option('display.max_columns', None)
df.head(20)

sample_row_n sample_row_n_name  rep  n_estimators  min_samples_leaf  \
0           NaN              None    0           100                 5   
1           NaN              None    0           100                 5   
2           NaN              None    0           100                 5   
3           NaN              None    0           100                 5   
4           NaN              None    1           100                 5   
5           NaN              None    1           100                 5   
6           NaN              None    1           100                 5   
7           NaN              None    1           100                 5   

   max_features include_raw model                                     fi  \
0          0.33         NaN    RF              LFI_absolute_sum_evaluate   
1          0.33       False    RF  LFI_absolute_sum_evaluate_without_raw   
2          0.33         NaN    RF                                   LIME   
3          0.33         NaN    RF                               TreeSHAP   
4          0.33         NaN    RF              LFI_absolute_sum_evaluate   
5          0.33       False    RF  LFI_absolute_sum_evaluate_without_raw   
6          0.33         NaN    RF                                   LIME   
7          0.33         NaN    RF                               TreeSHAP   

  splitting_strategy      fi_time  MSE_before_ablation  MSE_after_ablation_1  \
0         train-test    60.873118             0.663450              0.663450   
1         train-test    58.766427             0.663450              0.663450   
2         train-test  1121.099709             0.663450              0.660802   
3         train-test     0.191195             0.663450              0.674124   
4         train-test    66.080944             0.665677              0.665677   
5         train-test    36.440360             0.665677              0.665677   
6         train-test  1117.018032             0.665677              0.664571   
7         train-test     0.085452             0.665677              0.675571   

   MSE_after_ablation_2  MSE_after_ablation_3  MSE_after_ablation_4  \
0              0.663450              0.663726              0.663891   
1              0.663450              0.663726              0.663891   
2              0.659798              0.660763              0.660897   
3              0.685155              0.693433              0.700958   
4              0.665677              0.665677              0.665863   
5              0.665677              0.665677              0.665863   
6              0.663080              0.662973              0.661198   
7              0.685076              0.694941              0.698064   

   MSE_after_ablation_5  MSE_after_ablation_6  MSE_after_ablation_7  \
0              0.663721              0.663270              0.663099   
1              0.663721              0.663270              0.663099   
2              0.661164              0.661593              0.661163   
3              0.708959              0.713354              0.719238   
4              0.665863              0.665865              0.665865   
5              0.665863              0.665865              0.665865   
6              0.659864              0.659581              0.659298   
7              0.704361              0.712883              0.719092   

   MSE_after_ablation_8  MSE_after_ablation_9  MSE_after_ablation_10  \
0              0.663099              0.663145               0.663145   
1              0.663099              0.663145               0.663145   
2              0.662488              0.663076               0.663459   
3              0.724212              0.730834               0.732465   
4              0.665095              0.665095               0.665095   
5              0.665095              0.665095               0.665095   
6              0.659487              0.659574               0.659189   
7              0.723309              0.729261               0.731127   

 

In [10]:
# Select "fi_time" and "fi" columns
df[["fi","fi_time"]]

,fi,fi_time
0,LFI_absolute_sum_evaluate,60.873118
1,LFI_absolute_sum_evaluate_without_raw,58.766427
2,LIME,1121.099709
3,TreeSHAP,0.191195
4,LFI_absolute_sum_evaluate,66.080944
5,LFI_absolute_sum_evaluate_without_raw,36.440360
6,LIME,1117.018032
7,TreeSHAP,0.085452


In [4]:
n_testsize = df[['n', 'test_size']].drop_duplicates()

KeyError: "['n'] not in index"

In [ ]:
# methods = ["MDI_all_stumps", "MDI_sub_stumps", "MDI_all_stumps_without_raw", "MDI_sub_stumps_without_raw", "LFI_sum_absolute_all_stumps",
#            "LFI_absolute_sum_all_stumps", "LFI_sum_absolute_sub_stumps", "LFI_absolute_sum_sub_stumps", 
#            "LFI_sum_absolute_all_stumps_without_raw", "LFI_absolute_sum_all_stumps_without_raw",
#            "LFI_sum_absolute_sub_stumps_without_raw", "LFI_absolute_sum_sub_stumps_without_raw", "TreeSHAP", "LIME"]

methods = ["MDI_local_all_stumps_evaluate", "MDI_local_all_stumps_evaluate_without_raw","LFI_absolute_sum_evaluate","LFI_absolute_sum_evaluate_without_raw","TreeSHAP", "LIME"]
sample_row_n = df["n"].unique().tolist()
sample_row_n.sort()
train_test_map = {}
for i in range(len(sample_row_n)):
    train_test_map[sample_row_n[i]] = i
heritability = df["heritability"].unique().tolist()
heritability.sort()
nreps = df["rep"].max()
num_features = 10
n_testsize = df[['n', 'test_size']].drop_duplicates()
results = {}
for h in heritability:
    results[h] = {}
    for s in sample_row_n:
        results[h][s] = {}
        for m in methods:
            results[h][s][m] = {}
            for r in range(nreps+1):
                results[h][s][m][r] = []

In [ ]:
for h in heritability:
    for s in sample_row_n:
        for m in methods:
            for r in range(nreps+1):
                df_sub = df[(df["fi"] == m) & (df["n"] == s) & (df["heritability"] == h) & (df["rep"] == r)]
                assert len(df_sub["MSE_before_ablation"].unique()) == 1
                results[h][s][m][r].append(df_sub["MSE_before_ablation"].unique()[0])
                for i in range(num_features):
                    assert len(df_sub[f"MSE_after_ablation_{i+1}"].unique()) == 1
                    results[h][s][m][r].append(df_sub[f"MSE_after_ablation_{i+1}"].unique()[0])

In [ ]:
agg_results = {} 
for h in heritability:
    agg_results[h] = {}
    for s in sample_row_n:
        agg_results[h][s] = {}
        for m in methods:
            agg_results[h][s][m] = None

In [ ]:
for h in heritability:
    for s in sample_row_n:
        for m in methods:
            result = []
            for f in range(num_features+1):
                result.append(np.mean([results[h][s][m][r][f] for r in range(nreps+1)]))          
            agg_results[h][s][m] = result

In [ ]:
# Plotting
for i in range(len(heritability)):
    h = heritability[i]
    for j in range(len(sample_row_n)):
        s = sample_row_n[j]
        fig, ax = plt.subplots()
        for m in methods:
            if m in ["TreeSHAP", "LIME"]:
                ax.plot(range(num_features+1), agg_results[h][s][m], label=m, linestyle='dashed')
            else:
                ax.plot(range(num_features+1), agg_results[h][s][m], label=m)
        ax.set(xlabel='Number of features ablated', ylabel='MSE',
               title=f'Heritability = {h}, Train size = {s - n_testsize[n_testsize["n"] == s]["test_size"].values[0]}, Test size = {n_testsize[n_testsize["n"] == s]["test_size"].values[0]}')
        ax.legend()
        plt.show()